In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot the mse of the function f(x): x*\theta = 0
v = np.linspace(-1, 1, 100)
mse = v**2
plt.plot(v, mse)
plt.plot(v, v*2)
#use latex for the labels
plt.rcParams['text.usetex'] = True

plt.xlabel(r'$\theta$')
plt.ylabel(r'$E_{MSE}$')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics, optimizers
from matplotlib import pyplot as plt


data = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

train, test = data

train_data, train_labels = train
test_data, test_labels = test

train_labels = tf.one_hot(train_labels, 10)
test_labels = tf.one_hot(test_labels, 10)

plt.imshow(train_data[0], cmap='gray')
plt.show()


import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics

network = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Normalization(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

network2 = models.Sequential([
    #layers.Normalization(input_shape=(28, 28, 1)),
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (1,1), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

network.compile(loss = losses.CategoricalCrossentropy(), metrics=[metrics.CategoricalAccuracy()], optimizer=optimizers.SGD(learning_rate=0.001))

training = network.fit(train_data, train_labels, epochs=20, batch_size=128, validation_data=(test_data, test_labels))


In [ ]:
# show loss and accuracy in twin x plots
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(training.history['loss'], label='loss', color='red')
ax1.plot(training.history['val_loss'], label='val_loss', color='orange')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()
ax2.plot(training.history['categorical_accuracy'], label='accuracy', color='blue')	
ax2.plot(training.history['val_categorical_accuracy'], label='val_accuracy', color='green')
ax2.set_ylabel('accuracy')
ax2.legend()
plt.show()

In [ ]:
network.summary()


In [ ]:
import skimage
from support.bounding_boxes import create_bounding_boxes, draw_bounding_boxes

img = skimage.io.imread('images/Numbers.jpg', as_gray=True)
# resize image
img = skimage.transform.resize(img, (200, 300))

# increase contrast
#img = skimage.exposure.rescale_intensity(img, in_range=(0.2, 0.8))

plt.imshow(img, cmap='gray')
plt.show()

# do canny edge
edges = skimage.feature.canny(img, sigma=1)
plt.imshow(edges, cmap='gray')
plt.show()

# erosion
edges = skimage.morphology.binary_dilation(edges, footprint=np.ones((5,5)))
plt.imshow(edges, cmap='gray')
plt.show()

#label image
labels = skimage.measure.label(edges)

# create bounding boxes
rectangles = create_bounding_boxes(labels, square = True)
plt.imshow(img, cmap='gray')
draw_bounding_boxes(rectangles)
plt.show()
assigned_classes = []

# create bounding boxes with score
for rect in rectangles:
    x, y, w, h = rect
    # extract patch for rectangle
    patch = img[y:y+h, x:x+w]
    # resize patch to required size
    patch = skimage.transform.resize(patch, (28, 28))
    # invert colors
    patch = 1. - patch
    patch = np.where(patch < 0.5, 0, patch * 2 - 0.5)
    # get the network output for the patch
    output = network1(tf.expand_dims(patch * 255, 0))
    # get the predicted class
    label = np.argmax(output[0])
    assigned_classes.append(label)

plt.imshow(img, cmap='gray')
draw_bounding_boxes(rectangles, assigned_classes)
plt.show()